# Dijkstra: kürzeste Wege in einem Graphen

## Notwendige Bibliotheken

In [ ]:
import nrw_graph_unsave as ng
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

## Graph aus einer Datei einlesen

In [ ]:
df_staedte = pd.read_csv("Highway_Germany.txt", sep=",")
display(df_staedte)

In [ ]:
autobahn = ng.nrw_graph()
zeilen = df_staedte.shape[0]
for i in range(zeilen):
    source = df_staedte.iloc[i]['Start']
    target = df_staedte.iloc[i]['Ziel']
    dist = float(df_staedte.iloc[i]['Entfernung'])

    autobahn.fuegeKanteHinzu(source, target, gewicht=dist)
    autobahn.deflagKnoten(source)
    autobahn.deflagKnoten(target)

## Einige Hilfsfunktionen

In [ ]:
def entfernung (liste):
    return liste[2]

In [ ]:
def alleBesuchtenKnoten():
    alle = []
    for knoten in autobahn.alleKnoten():
        if autobahn.knotenHatFlag(knoten):
            (ueber, lang) = autobahn.getKnotenMarke(knoten)
            alle.append([knoten,ueber, lang])
    return alle

In [ ]:
def kantenVon(start):
    kanten = []
    for (s,z) in autobahn.alleKanten():
        if s == start:
            l = autobahn.kantenGewicht(s, z)
            kanten.append([s,z, l])
        elif z == start:
            l = autobahn.kantenGewicht(s, z)
            kanten.append([z, s, l])

    kanten.sort(key = entfernung)
    return kanten

In [ ]:
def schnittkantenDirektVon(start):
    kanten = []
    for (s,z) in autobahn.alleKanten():
        if s == start and not autobahn.knotenHatFlag(z):
            l = autobahn.kantenGewicht(s, z)
            kanten.append([s, z, l])
        elif z == start and not autobahn.knotenHatFlag(s):
            l = autobahn.kantenGewicht(s, z)
            kanten.append ([z,s, l])

    kanten.sort(key = entfernung)
    return kanten

In [ ]:
def alleSchnittkanten():
    kanten = []
    for (s,z) in autobahn.alleKanten():
        if autobahn.knotenHatFlag(s) and not autobahn.knotenHatFlag(z):
            l = autobahn.kantenGewicht(s, z)
            (ueber, weit) = autobahn.getKnotenMarke(s)
            l += weit
            kanten.append ([s, z, l])
        elif autobahn.knotenHatFlag(z) and not autobahn.knotenHatFlag(s):
            l = autobahn.kantenGewicht(z, s)
            (ueber, weit) = autobahn.getKnotenMarke(z)
            l += weit
            kanten.append([z, s, l])

    kanten.sort(key = entfernung)
    return kanten

## Von X nach Y

In [ ]:
#startknoten = "Berlin"
#zielknoten = "München"

In [ ]:
startknoten = "BIELEFELD - SENNESTADT (A 2)"
zielknoten = "KREUZ MUENSTER - NORD"

### Der Startknoten ist bereits besucht!

Zunächst eine Trivialität:
- Möchte man von X nach X reisen, so ist die kürzeste Verbindung über X mit einer Länge von 0.0

Also wird X mit einer Flagge und eine Marke der Form (ueber, laenge) versehen:

In [ ]:
autobahn.flagKnoten(startknoten)
autobahn.markiereKnoten(startknoten, (startknoten, 0.0))

Jetzt kann man sich alle geflaggten Knoten mit ihren Marken ansehen.

**Schau dir dazu die entsprechende Hilfsfunktion weiter oben an!**

In [ ]:
alleBesuchtenKnoten()

In [ ]:
alleSchnittkanten()

### Das Verfahren automatisiert

In [ ]:
while not autobahn.knotenHatFlag(zielknoten):
    sk = alleSchnittkanten()
    shortest = sk [0]
    print("Shortest =", shortest)
    ueber = shortest[0]
    neu = shortest[1]
    lang = shortest[2]
    autobahn.flagKnoten(neu)
    autobahn.markiereKnoten(neu, (ueber, lang))
    
besucht = {}
for kante in alleBesuchtenKnoten():
    besucht[kante[0]] = [kante[1], kante[2]]

node = zielknoten
ausgabe = ""
gesamt = besucht[zielknoten][1]
while node != startknoten:
    info = besucht[node]
    ueber = info[0]
    lang = autobahn.kantenGewicht(ueber, node)
    ausgabe = "Von " + ueber + " nach " + node + " (" + str(lang) + ")" +"\n" + ausgabe
    node = ueber
print("Es sind insgesamt", gesamt, "km")    
print (ausgabe)    
    

### Man hat mehr als gewünscht!

Man kann jetzt auch die kürzeste Route von Berlin zu jeder anderen Stadt berechnen, die im Laufe der Schritte markiert wurde.

In [ ]:
node = "KREUZ DORTMUND - NORDWEST"
ausgabe = ""
gesamt = besucht[node][1]
while node != startknoten:
    info = besucht[node]
    ueber = info[0]
    lang = autobahn.kantenGewicht(ueber, node)
    ausgabe = "Von " + ueber + " nach " + node + " (" + str(lang) + ")" +"\n" + ausgabe
    node = ueber
print("Es sind insgesamt", gesamt, "km")    
print (ausgabe)